In [ ]:
from typing import List
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    HumanMessage,
    SystemMessage,
)
from dotenv import load_dotenv
import os
from agent import CAMELAgent, get_sys_msgs
form src

In [ ]:

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
model_version = "gpt-3.5-turbo" #gpt-4
interviewer = "Joe Roegan"
interviewee = "Sam Altman"

## Init agents

In [ ]:
interviewer_sys_msg, interviewee_sys_msg = get_sys_msgs(
    interviewer, interviewee
)
interviewer_agent = CAMELAgent(interviewer_sys_msg, ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=model_version, temperature=0.2, model_kwargs={"frequency_penalty":0}))
interviewee_agent = CAMELAgent(interviewee_sys_msg, ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=model_version, temperature=0.2, model_kwargs={"frequency_penalty":0}))

interviewer_agent.reset()
interviewee_agent.reset()

conversation=[]
interviewee_msg = ""
interviewer_msg = ""

## Generation loop : run to generate a Q&A step

In [ ]:


step_msg = interviewer_agent.step(interviewee_msg)
interviewer_msg = HumanMessage(content=step_msg.content)
transcript = f"{interviewer}:\n\n{interviewer_msg.content}\n\n"
print(transcript)
conversation.append(transcript)


step_msg = interviewee_agent.step(interviewer_msg)
interviewee_msg = HumanMessage(content=step_msg.content)
transcript = f"{interviewee}:\n\n{interviewee_msg.content}\n\n"
print(transcript)
conversation.append(transcript)

## Director : here you can add directives to Joe

In [ ]:
directive = "you are going very angry if I'm calling you Jay again. You're name is Joe. You're proud of your name."

interviewer_agent.add_directives(directive)

## Director : here you can add directives to the interviewee

In [ ]:
directive = "Stop calling him Jay, call him Joe now"

interviewee_agent.add_directives(directive)

## Debug : check the messages of each participants

In [ ]:
print("Joe internal messages:")
for r in interviewer_agent.build_complete_message_list():
    print(f"===={r.type.capitalize()}:\n", r.content)

In [ ]:
print("interviewee internal messages:")
for r in interviewee_agent.build_complete_message_list():
    print(f"===={r.type.capitalize()}:\n", r.content)


## Get the full conversation

In [ ]:
conversation

In [26]:
with open('conversation.txt', 'w') as f:
    for line in conversation:
      f.write(line)


## Make it more conversational
(split the converation by using `conversation[n:m]` syntax)

In [ ]:
chat_llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=model_version, temperature=0, model_kwargs={"frequency_penalty":0})
prompt = "============\n"
for line in conversation[0:5]:
    prompt += line
prompt += "===========\n"
prompt += """
From the transcript above, enhance the conversation by making it more conversational : add some "hum", "yeah" and very little words as if the people were reacting quickly to each other, as if they were really talking.

for example :
BEFORE:
"Joe Roegan:
Sam. Everything's going good on my end, man. Just living the dream, you know? So, let's dive right into it. You've had quite the journey in the tech world, from starting your own company to leading Y Combinator. Can you tell us a bit about your background and how you got into the tech industry?"

AFTER :
"Joe Roegan:
Sam. Everything's going good on my end, man. Just living the dream, you know?

Sam Altman: yeah

Joe Roegan:
So, let's dive right into it. You've had quite the journey in the tech world, from starting your own company to leading Y Combinator.

Sam Altman: Mmh mmh

Joe Roegan: Can you tell us a bit about your background and how you got into the tech industry?
"""

new_conv = chat_llm([SystemMessage(content=prompt)])
print(new_conv.content)